In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/GraphRAG_fiscal')

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv('/content/drive/MyDrive/GraphRAG_fiscal/cles.env')

api_key = os.getenv("OPENAI_API_KEY")
token_github = os.getenv("GITHUB_TOKEN")

In [ ]:
!pip install langchain-text-splitter

ERROR: Could not find a version that satisfies the requirement langchain_text_splitter (from versions: none)
ERROR: No matching distribution found for langchain_text_splitter


In [ ]:
!pip install openai
!pip install PyPDF2

ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.3 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import json
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Chargement du PDF CGI
def load_cgi_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# 2. Segmentation adaptée au texte juridique
def segment_legal_text(text):
    splitter = RecursiveCharacterTextSplitter(
        separators=["\nArticle", "\nCHAPTER", "\nSECTION", "\n\n"],
        chunk_size=1000,
        chunk_overlap=200
    )
    return splitter.split_text(text)

In [ ]:
txt = load_cgi_pdf('cgi-2025-fr.pdf')
segments = segment_legal_text(txt)
segments[0]

" \nCODE GENERAL DES IMPOTS      \nInstitué par l’article 5 de la loi de finances  \nn° 43 -06 pour l’année budgétaire 2007, \npromulguée par le Dahir n° 1 -06-232 du 10 Hija \n1427 (31 décembre 2006),  \ntel que modifié et complété  \n \n \n \n \n \n \n \n \n2025 \n   \n3 \n PREAMBULE  \n \n    L'élaboration du Code Général des Impôts en 2007 s'inscrit dans le \nprolongement des r éformes menées par les pouvoirs publics suite aux \nHautes Instructions de Sa Majesté Le Roi Mohammed VI que Dieu \nL'Assiste  et tendant à l'amélioration de l'environnement juridique, fiscal et \néconomique de l'investissement par la modernisation des systèmes \nlégislatif, judiciaire et financier.  \n \n    Par aille urs, l'ouverture progressive de l'économie nationale \nconsécutive aux engagements pris par le Maroc vis -à-vis des instances \ninternationales et aux accords de libre -échange  signés notamment avec \nl'Union Européenne, les Etats -Unis d'Amérique, l'Egypte, la Jor danie et la \nTurquie néces

In [ ]:
len(segments)

493

In [ ]:
import os
import json
import csv
from typing import List, Dict, Any, Optional
from datetime import datetime

class ExtracteurFiscalMaroc:
    """
    Système d'extraction d'entités et de relations pour textes fiscaux marocains
    Support multiple: OpenAI
    """

    def __init__(self, provider: str = "openai", api_key: str = None, model: str = None):
        """
        Args:
            provider: "openai"
            api_key: Clé API
            model: Nom du modèle à utiliser
        """
        self.provider = provider.lower()
        self.api_key = api_key or os.getenv(f"{provider.upper()}_API_KEY")

        # Initialiser le client selon le provider
        if self.provider == "openai":
            from openai import OpenAI
            self.client = OpenAI(api_key=self.api_key)
            self.model = model or "gpt-4o-mini"

        else:
            raise ValueError(f"Provider non supporté: {provider}")

        # Types d'entités fiscales marocaines
        self.entity_types = [
            "LOI_FISCALE", "IMPOT", "TAUX", "CONTRIBUABLE",
            "REGIME_FISCAL", "OBLIGATION", "DECLARATION", "EXONERATION",
            "DEDUCTION", "ORGANISME", "DATE_LIMITE", "SANCTION",
            "SECTEUR_ACTIVITE", "SEUIL"
        ]

        # Types de relations fiscales
        self.relation_types = [
            "REGIT_PAR", "SOUMIS_A", "APPLIQUE_TAUX", "BENEFICIE_DE",
            "DOIT_DECLARER", "ECHEANCE", "ADMINISTRE_PAR", "SANCTIONNE_PAR",
            "REMPLACE", "CONDITIONNE", "CALCULE_PAR", "CONCERNE"
        ]

    def create_extraction_prompt(self, text: str) -> str:
        """Crée le prompt pour l'extraction sans afficher les types d'entités et relations"""
        return  f"""Tu es un expert en fiscalité marocaine. Analyse le texte suivant et extrais toutes les entités fiscales pertinentes ainsi que leurs relations, sans te limiter à des catégories pré-définies.

                TEXTE À ANALYSER:
                {text}

                INSTRUCTIONS:
                1. Identifie toutes les entités fiscales dans le texte.
                2. Détermine le type de chaque entité de manière descriptive (ex: impôt, taxe, contribution, etc.).
                3. Identifie toutes les relations entre les entités (ex: "lié à", "applicable sur", "calculé à partir de", etc.).
                4. Extrait les attributs pertinents (montants, pourcentages, dates, délais, etc.).

                Réponds UNIQUEMENT avec un JSON valide suivant ce format exact:
                {{
                "entities": [
                    {{
                    "id": "entite_1",
                    "text": "texte_entité",
                    "type": "TYPE_ENTITE",
                    "attributes": {{"attr1": "valeur1"}}
                    }}
                ],
                "relations": [
                    {{
                    "source_id": "entite_1",
                    "source_text": "texte source",
                    "target_id": "entite_2",
                    "target_text": "texte cible",
                    "type": "TYPE_RELATION",
                    "description": "description"
                    }}
                ]
                }}"""

    def extract_with_openai(self, text: str) -> Dict[str, Any]:
        """Extraction avec OpenAI"""
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Tu es un expert en droit fiscal marocain."},
                    {"role": "user", "content": self.create_extraction_prompt(text)}
                ],
                temperature=0.1,
                response_format={"type": "json_object"}
            )
            return json.loads(response.choices[0].message.content)
        except Exception as e:
            print(f"❌ Erreur OpenAI: {e}")
            return {"entities": [], "relations": []}




    def extract_entities_relations(self, text: str) -> Dict[str, Any]:
        """
        Extrait les entités et relations selon le provider configuré
        """
        print(f"🔄 Extraction avec {self.provider.upper()} ({self.model})...")

        if self.provider == "openai":
            return self.extract_with_openai(text)
        elif self.provider == "anthropic":
            return self.extract_with_anthropic(text)
        elif self.provider == "ollama":
            return self.extract_with_ollama(text)
        else:
            return {"entities": [], "relations": []}

    def save_entities_to_csv(self, entities: List[Dict], filename: str = "entites_fiscales.csv"):
        """Enregistre les entités en CSV"""
        if not entities:
            print("⚠️  Aucune entité à enregistrer")
            return

        all_attributes = set()
        for entity in entities:
            if "attributes" in entity:
                all_attributes.update(entity["attributes"].keys())

        fieldnames = ["id", "text", "type"] + sorted(all_attributes)

        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for entity in entities:
                row = {
                    "id": entity.get("id", ""),
                    "text": entity.get("text", ""),
                    "type": entity.get("type", "")
                }
                if "attributes" in entity:
                    for attr, value in entity["attributes"].items():
                        row[attr] = str(value) if value is not None else ""

                writer.writerow(row)

        print(f"✅ {len(entities)} entités → '{filename}'")

    def save_relations_to_csv(self, relations: List[Dict], filename: str = "relations_fiscales.csv"):
        """Enregistre les relations en CSV"""
        if not relations:
            print("⚠️  Aucune relation à enregistrer")
            return

        fieldnames = ["source_id", "source_text", "target_id", "target_text", "type", "description"]

        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for relation in relations:
                row = {
                    "source_id": relation.get("source_id", relation.get("source", "")),
                    "source_text": relation.get("source_text", ""),
                    "target_id": relation.get("target_id", relation.get("target", "")),
                    "target_text": relation.get("target_text", ""),
                    "type": relation.get("type", ""),
                    "description": relation.get("description", "")
                }
                writer.writerow(row)

        print(f"✅ {len(relations)} relations → '{filename}'")

    def process_and_save(self, text: str,
                         entities_file: str = "entites_fiscales.csv",
                         relations_file: str = "relations_fiscales.csv"):
        """Traite un texte et enregistre en CSV"""

        result = self.extract_entities_relations(text)

        entities = result.get("entities", [])
        relations = result.get("relations", [])

        print(f"\n📊 Résultats:")
        print(f"   • {len(entities)} entités")
        print(f"   • {len(relations)} relations")

        if entities or relations:
            print(f"\n💾 Enregistrement...")
            self.save_entities_to_csv(entities, entities_file)
            self.save_relations_to_csv(relations, relations_file)
            print(f"\n✨ Extraction terminée avec succès!")

        return {
            "entities_count": len(entities),
            "relations_count": len(relations),
            "entities_file": entities_file,
            "relations_file": relations_file
        }

In [ ]:
extracteur = ExtracteurFiscalMaroc(
    provider="openai",
    api_key=api_key,
    model="gpt-4o-mini"
)

In [ ]:
result = extracteur.extract_entities_relations(segments[0])
print(json.dumps(result, indent=2, ensure_ascii=False))

🔄 Extraction avec OPENAI (gpt-4o-mini)...
{
  "entities": [
    {
      "id": "entite_1",
      "text": "Code Général des Impôts",
      "type": "Code",
      "attributes": {
        "année": "2007"
      }
    },
    {
      "id": "entite_2",
      "text": "loi de finances n° 43-06",
      "type": "Loi",
      "attributes": {
        "année": "2007"
      }
    },
    {
      "id": "entite_3",
      "text": "Dahir n° 1-06-232",
      "type": "Dahir",
      "attributes": {
        "date": "31 décembre 2006"
      }
    },
    {
      "id": "entite_4",
      "text": "impôt sur les sociétés (I.S)",
      "type": "Impôt",
      "attributes": {}
    },
    {
      "id": "entite_5",
      "text": "impôt sur le revenu (I.R)",
      "type": "Impôt",
      "attributes": {}
    },
    {
      "id": "entite_6",
      "text": "Taxe sur la valeur ajoutée (T.V.A)",
      "type": "Taxe",
      "attributes": {}
    },
    {
      "id": "entite_7",
      "text": "Droits d'enregistrement (D.E)",
      

In [ ]:
resultats_finaux

{'metadata': {'date_extraction': '2025-11-17T00:32:45.017482',
  'total_segments': 493,
  'provider': 'openai',
  'model': 'gpt-4o-mini'},
 'segments': [{'segment_id': 0,
   'segment_text_preview': ' \nCODE GENERAL DES IMPOTS      \nInstitué par l’article 5 de la loi de finances  \nn° 43 -06 pour l’année budgétaire 2007, \npromulguée par le Dahir n° 1...',
   'segment_length': 5374,
   'timestamp': '2025-11-17T00:33:43.135467',
   'statistiques': {'nb_entites': 19, 'nb_relations': 18},
   'resultats': {'entities': [{'id': 'entite_1',
      'text': 'Code Général des Impôts',
      'type': 'Code',
      'attributes': {'année': '2007'}},
     {'id': 'entite_2',
      'text': 'Dahir n° 1-06-232',
      'type': 'Dahir',
      'attributes': {'date': '31 décembre 2006'}},
     {'id': 'entite_3',
      'text': 'loi de finances n° 43-06',
      'type': 'Loi de finances',
      'attributes': {'année': '2007'}},
     {'id': 'entite_4',
      'text': 'impôt sur les sociétés (I.S)',
      'type': '

In [ ]:
import json
from tqdm import tqdm  # Pour la barre de progression
import time

# Structure pour stocker tous les résultats
resultats_finaux = {
    "metadata": {
        "date_extraction": datetime.now().isoformat(),
        "total_segments": len(segments),
        "provider": extracteur.provider,
        "model": extracteur.model
    },
    "segments": []
}

print("🔄 Début de l'extraction des entités fiscales...")

for i, segment in enumerate(tqdm(segments)):
    print(f"\n📄 Traitement du segment {i+1}/{len(segments)}")
    print(f"Longueur du segment: {len(segment)} caractères")

    try:
        # Extraction pour ce segment
        resultat = extracteur.extract_entities_relations(segment)

        # Structure détaillée pour chaque segment
        segment_resultat = {
            "segment_id": i,
            "segment_text_preview": segment[:150] + "..." if len(segment) > 150 else segment,
            "segment_length": len(segment),
            "timestamp": datetime.now().isoformat(),
            "statistiques": {
                "nb_entites": len(resultat.get('entities', [])),
                "nb_relations": len(resultat.get('relations', []))
            },
            "resultats": resultat
        }

        resultats_finaux["segments"].append(segment_resultat)

        # Affichage des résultats du segment
        print(f"✅ Segment {i+1}: {segment_resultat['statistiques']['nb_entites']} entités, {segment_resultat['statistiques']['nb_relations']} relations")

        # Sauvegarde intermédiaire (au cas où)
        if i % 20 == 0:  # Tous les 20 segments
            with open(f"resultats_intermediaire_{i}.json", "w", encoding="utf-8") as f:
                json.dump(resultats_finaux, f, indent=2, ensure_ascii=False)
            print(f"💾 Sauvegarde intermédiaire à l'index {i}")

        # Petite pause pour éviter les rate limits
        time.sleep(0.5)

    except Exception as e:
        print(f"❌ Erreur sur le segment {i+1}: {e}")

        # Enregistrer l'erreur dans les résultats
        segment_erreur = {
            "segment_id": i,
            "segment_text_preview": segment[:150] + "..." if len(segment) > 150 else segment,
            "segment_length": len(segment),
            "timestamp": datetime.now().isoformat(),
            "erreur": str(e),
            "statistiques": {
                "nb_entites": 0,
                "nb_relations": 0
            },
            "resultats": {"entities": [], "relations": []}
        }

        resultats_finaux["segments"].append(segment_erreur)

# Calcul des statistiques finales
total_entites = sum(seg["statistiques"]["nb_entites"] for seg in resultats_finaux["segments"])
total_relations = sum(seg["statistiques"]["nb_relations"] for seg in resultats_finaux["segments"])
segments_reussis = sum(1 for seg in resultats_finaux["segments"] if "erreur" not in seg)

resultats_finaux["metadata"].update({
    "total_entites_extractees": total_entites,
    "total_relations_extractees": total_relations,
    "segments_traites_avec_succes": segments_reussis,
    "segments_en_erreur": len(segments) - segments_reussis,
    "taux_reussite": f"{(segments_reussis/len(segments))*100:.1f}%"
})

# Sauvegarde FINALE en JSON
nom_fichier_json = f"extraction_complete_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(nom_fichier_json, "w", encoding="utf-8") as f:
    json.dump(resultats_finaux, f, indent=2, ensure_ascii=False, ensure_ascii=False)

print(f"\n✨ Extraction terminée!")
print(f"📊 Statistiques finales:")
print(f"   • {len(resultats_finaux['segments'])} segments traités")
print(f"   • {total_entites} entités extraites au total")
print(f"   • {total_relations} relations extraites au total")
print(f"   • {segments_reussis} segments réussis")
print(f"   • {len(segments) - segments_reussis} segments en erreur")
print(f"💾 Fichier JSON sauvegardé: '{nom_fichier_json}'")

# Optionnel: Sauvegarde en CSV également
try:
    # Consolider toutes les entités et relations
    toutes_entites = []
    toutes_relations = []

    for segment in resultats_finaux["segments"]:
        if "resultats" in segment:
            toutes_entites.extend(segment["resultats"].get("entities", []))
            toutes_relations.extend(segment["resultats"].get("relations", []))

    # Sauvegarder en CSV
    extracteur.save_entities_to_csv(toutes_entites, "entites_consolidees.csv")
    extracteur.save_relations_to_csv(toutes_relations, "relations_consolidees.csv")

    print(f"📁 Fichiers CSV sauvegardés: 'entites_consolidees.csv', 'relations_consolidees.csv'")

except Exception as e:
    print(f"⚠️ Erreur lors de la sauvegarde CSV: {e}")

return resultats_finaux

🔄 Début de l'extraction des entités fiscales...


  0%|          | 0/493 [00:00<?, ?it/s]


📄 Traitement du segment 1/493
Longueur du segment: 5374 caractères
🔄 Extraction avec OPENAI (gpt-4o-mini)...
✅ Segment 1: 19 entités, 18 relations
💾 Sauvegarde intermédiaire à l'index 0


  0%|          | 1/493 [00:58<8:00:40, 58.62s/it]


📄 Traitement du segment 2/493
Longueur du segment: 238 caractères
🔄 Extraction avec OPENAI (gpt-4o-mini)...
✅ Segment 2: 3 entités, 3 relations


...


📄 Traitement du segment 126/493
Longueur du segment: 1536 caractères
🔄 Extraction avec OPENAI (gpt-4o-mini)...
❌ Erreur OpenAI: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-s78Wk2BH28NnlaT3UFjnr2zg on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
✅ Segment 126: 0 entités, 0 relations


 26%|██▌       | 126/493 [56:08<2:21:13, 23.09s/it]


📄 Traitement du segment 127/493
Longueur du segment: 1661 caractères
🔄 Extraction avec OPENAI (gpt-4o-mini)...


 26%|██▌       | 126/493 [56:28<2:44:30, 26.90s/it]


KeyboardInterrupt: 